In [1]:
#!usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
from librosa.core import load, resample, stft, istft
from librosa.output import write_wav
from librosa.util import find_files
import numpy as np
import const_for_colab as C
import util_for_colab as util
import yaml
import os

## 周辺知識
- **パラデータ** ... 各トラック単体のデータ
- **ステムデータ** ... 複数のトラックをまとめたもの(パラデータの集合)である．

MedleyDBのデータセットには生音源と，各種のステム音源があり，Metadataのyamlファイルにそれぞれの情報が記載されている．
vocalが入っているのは，instrumentalの属性値が'male singer'，'female singer'，'vocalists'のものである．複数ある場合もあれば，どれか一つのみの場合もあるので各々でマッチさせる必要がある．

In [3]:
PATH_MEDLEYDB = 'E:\Datasets\sound\MedleyDB\V1'
PATH_E_DRIVE = 'E:/Datasets/sound/MedleyDB_gen/'

metadata_list = find_files(PATH_MEDLEYDB.rstrip('\V1'), ext='yaml')
print(metadata_list[:5])

['E:\\Datasets\\sound\\MedleyDB\\Metadata\\AClassicEducation_NightOwl_METADATA.yaml', 'E:\\Datasets\\sound\\MedleyDB\\Metadata\\AimeeNorwich_Child_METADATA.yaml', 'E:\\Datasets\\sound\\MedleyDB\\Metadata\\AimeeNorwich_Flying_METADATA.yaml', 'E:\\Datasets\\sound\\MedleyDB\\Metadata\\AlexanderRoss_GoodbyeBolero_METADATA.yaml', 'E:\\Datasets\\sound\\MedleyDB\\Metadata\\AlexanderRoss_VelvetCurtain_METADATA.yaml']


In [4]:
def save_medleydb_mix(y_mixture, file_name):
    mix_spec = np.abs(
                    stft(y_mixture, n_fft=C.FFT_SIZE, hop_length=C.HOP_LENGTH))\
                .astype(np.float32)
    
    phase = np.exp(1.j*np.angle(mix_spec))
    y_mix = istft(mix_spec*phase,
                  hop_length=C.HOP_LENGTH, win_length=C.FFT_SIZE)
    
    write_wav(PATH_E_DRIVE + file_name + '_mix.wav', y_mix, C.SAMPLING_RATE)
    print('Saving: ' + PATH_E_DRIVE + file_name + '_mix.wav')

def save_medleydb_inst(y_instrument, file_name):
    inst_spec = np.abs(
                    stft(y_instrument, n_fft=C.FFT_SIZE, hop_length=C.HOP_LENGTH))\
                .astype(np.float32)
    phase = np.exp(1.j*np.angle(inst_spec))
    y_inst = istft(inst_spec*phase,
                   hop_length=C.HOP_LENGTH, win_length=C.FFT_SIZE)
    write_wav(PATH_E_DRIVE + file_name + '_inst.wav', y_inst, C.SAMPLING_RATE)
    print('Saving: ' + PATH_E_DRIVE + file_name + '_inst.wav')

def save_medleydb_vocal(y_vocal, file_name):
    vocal_spec = np.abs(
                    stft(y_vocal, n_fft=C.FFT_SIZE, hop_length=C.HOP_LENGTH))\
                .astype(np.float32)
    phase = np.exp(1.j*np.angle(vocal_spec))
    y_vocal = istft(vocal_spec*phase,
                    hop_length=C.HOP_LENGTH, win_length=C.FFT_SIZE)
    write_wav(PATH_E_DRIVE + file_name + '_vocal.wav', y_vocal, C.SAMPLING_RATE)
    print('Saving: ' + PATH_E_DRIVE + file_name + '_vocal.wav')

In [5]:
for metafile in metadata_list:
    print('YAML file: %s' % metafile)
    
    # 曲名を抽出(末尾の'_METADATA.yaml'を除いた文字列を格納)
    songname = metafile.split('\\')[-1][:-14]
    print('song: %s' % songname)
    
    # yamlファイルの読み込み
    with open(metafile, mode='r') as f:
        data = yaml.load(f)
    
    # インストがあるものはスキップ
    if data['instrumental'] != 'no':
        print('Instrumental track. Skipped.')
        continue
    
    stem_vocal = []
    stem_inst = []
    stem_path = os.path.join(PATH_MEDLEYDB, songname, data['stem_dir'])
    mix_filename = data["mix_filename"]
    
    # stemデータの抽出
    for s in data['stems']:
        stem = data['stems'][s]
        file_name = stem['filename']
        
        # 各ステムファイルのcomponent:構成要素とinstrument:楽器を表示
        print('stem: %s %s %s' % (file_name, stem['component'], stem['instrument']))
        
        # instrumentの属性値がボーカルのものと楽器のものを分ける
        if 'male' in stem['instrument'] or \
            'singer' in stem['instrument'] or \
            'vocalists' in stem['instrument']:
                stem_vocal.append(file_name)
                print('this is vocal!')
        else:
            stem_inst.append(file_name)
        
    print('detexted vocals:')
    print(stem_vocal)
    
    # ボーカルもインストもないものはスキップ
    if (len(stem_vocal) == 0) or (len(stem_inst) == 0):
        print('empty vocal or inst data ... skip!')
        continue
    
    # 各音源をまとめる
    audio_inst = sum([load(os.path.join(stem_path, f), sr=None, mono=True)[0]
                      for f in stem_inst])
    audio_vocal = sum([load(os.path.join(stem_path, f), sr=None, mono=True)[0]
                      for f in stem_vocal])
#     audio_mix = audio_inst + audio_vocal
    audio_mix, _ = load(os.path.join(PATH_MEDLEYDB, songname, mix_filename),
                            sr=None, mono=True)
    
#     # 各音源を保存
#     # リサンプリング
#     audio_mix = resample(audio_mix, 44100, C.SAMPLING_RATE)
#     audio_inst = resample(audio_inst, 44100, C.SAMPLING_RATE)
#     audio_vocal = resample(audio_vocal, 44100, C.SAMPLING_RATE)
#     save_medleydb_mix(audio_mix, songname)
#     save_medleydb_inst(audio_inst, songname)
#     save_medleydb_vocal(audio_vocal, songname)
    
    # スペクトログラムを保存
    util.save_spectrogram(audio_mix, audio_inst, audio_vocal, songname)
    
    print('*' * 150)

YAML file: E:\Datasets\sound\MedleyDB\Metadata\AClassicEducation_NightOwl_METADATA.yaml
song: AClassicEducation_NightOwl
stem: AClassicEducation_NightOwl_STEM_11.wav  tambourine
stem: AClassicEducation_NightOwl_STEM_13.wav melody male singer
this is vocal!
stem: AClassicEducation_NightOwl_STEM_09.wav  synthesizer
stem: AClassicEducation_NightOwl_STEM_07.wav  distorted electric guitar
stem: AClassicEducation_NightOwl_STEM_08.wav melody male singer
this is vocal!
stem: AClassicEducation_NightOwl_STEM_04.wav melody distorted electric guitar
stem: AClassicEducation_NightOwl_STEM_03.wav  distorted electric guitar
stem: AClassicEducation_NightOwl_STEM_02.wav  drum set
stem: AClassicEducation_NightOwl_STEM_06.wav  distorted electric guitar
stem: AClassicEducation_NightOwl_STEM_10.wav  vocalists
this is vocal!
stem: AClassicEducation_NightOwl_STEM_01.wav bass electric bass
stem: AClassicEducation_NightOwl_STEM_12.wav  fx/processed sound
stem: AClassicEducation_NightOwl_STEM_05.wav  clean elect

(513, 9857)
(513, 9857)
(513, 9857)
Saving: E:/Datasets/sound/MedleyDB/spectrograms/BrandonWebster_DontHearAThing.npz
******************************************************************************************************************************************************
YAML file: E:\Datasets\sound\MedleyDB\Metadata\BrandonWebster_YesSirICanFly_METADATA.yaml
song: BrandonWebster_YesSirICanFly
stem: BrandonWebster_YesSirICanFly_STEM_02.wav melody male singer
this is vocal!
stem: BrandonWebster_YesSirICanFly_STEM_01.wav  piano
detexted vocals:
['BrandonWebster_YesSirICanFly_STEM_02.wav']
(513, 5722)
(513, 5722)
(513, 5722)
Saving: E:/Datasets/sound/MedleyDB/spectrograms/BrandonWebster_YesSirICanFly.npz
******************************************************************************************************************************************************
YAML file: E:\Datasets\sound\MedleyDB\Metadata\CelestialShore_DieForUs_METADATA.yaml
song: CelestialShore_DieForUs
stem: CelestialShore_DieFo

(513, 12836)
(513, 12836)
(513, 12836)
Saving: E:/Datasets/sound/MedleyDB/spectrograms/Debussy_LenfantProdigue.npz
******************************************************************************************************************************************************
YAML file: E:\Datasets\sound\MedleyDB\Metadata\DreamersOfTheGhetto_HeavyLove_METADATA.yaml
song: DreamersOfTheGhetto_HeavyLove
stem: DreamersOfTheGhetto_HeavyLove_STEM_09.wav  synthesizer
stem: DreamersOfTheGhetto_HeavyLove_STEM_07.wav  synthesizer
stem: DreamersOfTheGhetto_HeavyLove_STEM_08.wav  female singer
this is vocal!
stem: DreamersOfTheGhetto_HeavyLove_STEM_04.wav  auxiliary percussion
stem: DreamersOfTheGhetto_HeavyLove_STEM_03.wav  fx/processed sound
stem: DreamersOfTheGhetto_HeavyLove_STEM_02.wav  vocalists
this is vocal!
stem: DreamersOfTheGhetto_HeavyLove_STEM_06.wav melody male singer
this is vocal!
stem: DreamersOfTheGhetto_HeavyLove_STEM_01.wav bass electric bass
stem: DreamersOfTheGhetto_HeavyLove_STEM_05.wa

(513, 16275)
(513, 16275)
(513, 16275)
Saving: E:/Datasets/sound/MedleyDB/spectrograms/HopAlong_SisterCities.npz
******************************************************************************************************************************************************
YAML file: E:\Datasets\sound\MedleyDB\Metadata\InvisibleFamiliars_DisturbingWildlife_METADATA.yaml
song: InvisibleFamiliars_DisturbingWildlife
stem: InvisibleFamiliars_DisturbingWildlife_STEM_09.wav melody male singer
this is vocal!
stem: InvisibleFamiliars_DisturbingWildlife_STEM_07.wav melody vocalists
this is vocal!
stem: InvisibleFamiliars_DisturbingWildlife_STEM_08.wav melody vocalists
this is vocal!
stem: InvisibleFamiliars_DisturbingWildlife_STEM_04.wav  drum set
stem: InvisibleFamiliars_DisturbingWildlife_STEM_03.wav melody trumpet section
stem: InvisibleFamiliars_DisturbingWildlife_STEM_02.wav  claps
stem: InvisibleFamiliars_DisturbingWildlife_STEM_06.wav  shaker
stem: InvisibleFamiliars_DisturbingWildlife_STEM_10.wav

MemoryError: 